In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm
import seaborn as sns
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid")
import csv
import time

import scipy.stats as stats
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load dữ liệu từ Kaggle:

In [ ]:
df_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
df_train.head()

In [ ]:
df_test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
df_test.head()

In [ ]:
shops=df_train['shop_id'].unique().tolist()
print(shops)

Tạo một dict để chứa các dòng dữ liệu của từng shop

In [ ]:
shopdf={}
for shop in shops:
    shopdf[shop]=pd.DataFrame(df_train.loc[df_train['shop_id'] == shop])
#print(shopdf)

Với mỗi shop, đếm số lượng bán được của mỗi item theo tháng

In [ ]:
shopdf_agg={}
for shop in shops:
    s=shopdf[shop].groupby(['date_block_num','item_id'])['item_cnt_day'].agg(item_cnt_agg='sum')
    s['date_block_num'] = s.index.get_level_values('date_block_num') 
    s['item_id'] = s.index.get_level_values('item_id') 
    s.reset_index(drop=True, inplace=True)
    s = s.reindex(['date_block_num','item_id','item_cnt_agg'], axis=1)
    shopdf_agg[shop]=s

Do nhiều item có những tháng không bán được sản phẩm, ta thêm các dòng dữ liệu số sản phẩm bán được là 0.

In [ ]:
data_ideal_df = [[0, 0], [1, 0],[2, 0],[3, 0],[4, 0],[5, 0],[6, 0],[7, 0],[8, 0],
                 [9, 0],[10, 0],[11, 0],[12, 0],[13, 0],[14, 0],[15, 0],[16, 0],[17, 0],[18, 0],
                 [19, 0],[20, 0],[21, 0],[22, 0],[23, 0],[24, 0],[25, 0],[26, 0],[27, 0],[28, 0],
                 [29, 0],[30, 0],[31, 0],[32, 0],[33, 0]] 
ideal_df = pd.DataFrame(data_ideal_df, columns = ['date_block_num', 'item_id'])

shop_item={}
sh_itm_mn={}
model=[]
for shop in shops:
    shop_item[shop]=shopdf_agg[shop]['item_id'].unique().tolist()
    for item in shop_item[shop]:
        mod=str(int(shop))+'_'+str(int(item))

        model.append(mod)
        temp = pd.DataFrame(shopdf_agg[shop].loc[shopdf_agg[shop]['item_id']==item])
        ideal_df['item_id'] = item  
        
        # merging the two dataframes so that every item contains every dateblocknum even if it is zero
        sh_itm_mn[mod]= pd.merge(temp, ideal_df, on=['date_block_num', 'item_id'], how='outer', sort= True).fillna(0)

Tạo các model theo từng cặp shopid_itemid

In [ ]:
# with statsmodels
shop_item_statsmodel={}
for i in model:
    X=sh_itm_mn[i]['date_block_num']
    Y=sh_itm_mn[i]['item_cnt_agg']
    X = sm.add_constant(X, has_constant='add') # adding a constant
    shop_item_statsmodel[i] = sm.OLS(Y, X).fit()
    #predictions = shop_item_statsmodel[i].predict(X) 
    #print_model = shop_item_statsmodel[i].summary()

Predict và lưu kết quả

In [ ]:
test_mod=[]
monthToPredict=34

filename='Submission.csv'
with open(filename,'a+') as csvFile:
    writer = csv.writer(csvFile)
    header=['ID','item_cnt_month']
    writer.writerow(header)
    for index, row in df_test.iterrows():
        shop_test=row['shop_id']
        item_test=row['item_id']
        mod=str(int(shop_test))+'_'+str(int(item_test))
        test_mod.append(mod)
        if mod not in model:
            row=[index,0]
        else:
            month=pd.DataFrame([[monthToPredict]],columns=['date_block'])
            X = sm.add_constant(month, has_constant='add')
            raw_pred=shop_item_statsmodel[mod].predict(X)
            if raw_pred[0] < 0:
                pred=0
            else:
                pred=raw_pred[0]
            predictedValue=pred
            row=[index,predictedValue]
        writer.writerow(row)
csvFile.close()  